In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import torchvision 

In [2]:
from torch.utils.data import Dataset 
import cv2
import matplotlib.pyplot as plt
from xml.dom import minidom
from os.path import basename
import glob

In [3]:
def unpickle(file):
   import pickle
   with open(file, 'rb') as fo:
      dict = pickle.load(fo, encoding='bytes')
   return dict

In [ ]:
training=torchvision.datasets.ImageNet(root='/kaggle/input/imagenet-object-localization-challenge/ILSVRC', train=True, download=True)

In [4]:
def generate_noise(path,t,ct):
    img=cv2.imread(path)
    img=np.array(img)
    img=img/255.0
    
    epsilon=np.random.randn(*img.shape).astype(np.float32)
    noise = np.exp(-t / ct)
    noise=np.clip(noise,0,1)
    new_img=np.sqrt(noise)*img + np.sqrt(1-noise)*epsilon
    new_img=np.clip(new_img,0,1)
    
    return noise

In [5]:
xml_folder=r'/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Annotations/CLS-LOC/train'
train_folder=r'/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/*'

In [6]:
train_folders = glob.glob('/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/*')

In [7]:
folder=(train_folders[0]+'/*')

In [8]:
folder_name=glob.glob(folder)

In [9]:
(folder_name)

['/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_3090.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_6545.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_1438.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_2277.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_9740.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_1930.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_7506.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n01531178_1318.JPEG',
 '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/n01531178/n0153

In [ ]:
for i in range(1,101):
    generate_noise(folder_name[0],0.0001,0.02,i,100)

In [37]:
class Diffusion(nn.Module):
    def __init__(self,batch) -> None:
        super().__init__()
        self.batch=batch
        self.time_embed = nn.Linear(1, 32).float()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
 
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=(3,3), stride=1, padding=1)

    def forward(self,x,t):

        t=t.view(self.batch,1).float()
        x = self.act1(self.conv1(x))
        x = self.drop1(x)
        t_emb = self.time_embed(t).unsqueeze(-1).unsqueeze(-1)
        x= x + t_emb
        # input 32x32x32, output 32x32x32
        x = self.act2(self.conv2(x))
        x = x+ t_emb
        x = self.conv3(x)
        return x

model = Diffusion(1)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.1)
def generate(path,t,ct):
    img=cv2.imread(path)
    img=np.array(img)
    img=img/255.0
    
    epsilon=np.random.randn(*img.shape).astype(np.float32)
    noise = np.exp(-t / ct)
    noise=np.clip(noise,0,1)
    epsilon=np.clip(epsilon,0,1)
    new_img=np.sqrt(noise)*img + np.sqrt(1-noise)*epsilon
    new_img=np.clip(new_img,0,1)
    return torch.tensor(new_img,dtype=torch.float32).permute(2,0,1).unsqueeze(0), torch.tensor(epsilon,dtype=torch.float32).permute(2,0,1).unsqueeze(0)

def noising(path,t,epoch):
    new_img,error=generate(path,t,epoch)
    timestep = torch.tensor([t/epoch],dtype=torch.float32) 
    pred_error=model.forward(new_img,timestep)
    loss = loss_fn(pred_error, error)
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
def train(folder,epoch):
    for e in range(10):
        path=folder[e]
        print(path)
        for t in range(epoch):
            noising(path,t,epoch)
        

In [ ]:
train(folder_name,100)